# 🧠 Création des Embeddings

Génération des vecteurs d'embeddings avec **Mistral AI** ou **sentence-transformers**.

**Étapes :**

1. Configuration du provider
2. Génération des vecteurs
3. Validation et analyse
4. Export pour FAISS


## 1️⃣ Configuration


In [1]:
import json
import numpy as np
from pathlib import Path
from typing import List

from tqdm.notebook import tqdm

from src.config.constants import PROCESSED_DATA_DIR
from src.config.settings import settings

print("✅ Configuration OK")
print(f"📊 Provider d'embeddings : {settings.embedding_provider}")
print(f"🧠 Modèle : {settings.embedding_model}")

✅ Configuration OK
📊 Provider d'embeddings : mistral
🧠 Modèle : mistral-embed


## 2️⃣ Chargement des Données


In [2]:
# Charger les documents prétraités
documents_path = PROCESSED_DATA_DIR / "rag_documents.json"

with open(documents_path, "r", encoding="utf-8") as f:
    documents = json.load(f)

print(f"📊 {len(documents)} documents chargés")
print(f"\n📋 Exemple de contenu à vectoriser :")
print(documents[0]["content"][:300])

📊 497 documents chargés

📋 Exemple de contenu à vectoriser :
Titre: Formation Civique et Citoyenne "Cadre de vie et architecture"
Ville: Marseille
Date: Du 14/06/2022 à 07:00 au 31/12/2025 à 22:59
Description: Formation obligatoire à l'attention des volontaires de Service Civique, proposée par la MAV PACA.
Adresse: 12 Bd Théodore Thurner 13006 Marseille


## 3️⃣ Classe d'Embeddings Unifiée


In [3]:
from mistralai import Mistral
from sentence_transformers import SentenceTransformer


class EmbeddingGenerator:
    """Générateur d'embeddings unifié (Mistral ou SentenceTransformers)."""

    def __init__(self, provider: str = None, model_name: str = None, api_key: str = None):
        self.provider = provider or settings.embedding_provider
        self.model_name = model_name or settings.embedding_model

        print(f"🔄 Initialisation du provider : {self.provider}")

        if self.provider == "mistral":
            if not api_key and not settings.mistral_api_key:
                raise ValueError("Mistral API key required")

            self.client = Mistral(api_key=api_key or settings.mistral_api_key)
            self.dimension = 1024
            print(f"✅ Mistral client initialisé (dimension: {self.dimension})")

        else:  # sentence-transformers
            self.model = SentenceTransformer(self.model_name)
            self.dimension = self.model.get_sentence_embedding_dimension()
            print(f"✅ SentenceTransformer chargé (dimension: {self.dimension})")

    def encode(
        self, texts: List[str], batch_size: int = 32, show_progress: bool = True
    ) -> np.ndarray:
        """Encode une liste de textes en embeddings."""

        if self.provider == "mistral":
            return self._encode_mistral(texts, batch_size, show_progress)
        else:
            return self._encode_sentence_transformer(texts, batch_size, show_progress)

    def _encode_mistral(self, texts: List[str], batch_size: int, show_progress: bool) -> np.ndarray:
        """Encode avec Mistral API."""
        embeddings = []

        iterator = range(0, len(texts), batch_size)
        if show_progress:
            iterator = tqdm(iterator, desc="🧠 Mistral Embeddings")

        for i in iterator:
            batch = texts[i : i + batch_size]

            # Appel API Mistral
            response = self.client.embeddings.create(model=self.model_name, inputs=batch)

            # Extraire les embeddings
            batch_embeddings = [item.embedding for item in response.data]
            embeddings.extend(batch_embeddings)

        return np.array(embeddings, dtype=np.float32)

    def _encode_sentence_transformer(
        self, texts: List[str], batch_size: int, show_progress: bool
    ) -> np.ndarray:
        """Encode avec SentenceTransformer."""
        return self.model.encode(
            texts, batch_size=batch_size, show_progress_bar=show_progress, convert_to_numpy=True
        )


print("✅ Classe EmbeddingGenerator définie")

✅ Classe EmbeddingGenerator définie


## 4️⃣ Génération des Embeddings


In [4]:
# Initialiser le générateur
generator = EmbeddingGenerator()

# Extraire les contenus
contents = [doc["content"] for doc in documents]

print(f"🧠 Génération de {len(contents)} embeddings...")
print("⚠️ Cela peut prendre plusieurs minutes\n")

# Générer les embeddings
embeddings = generator.encode(contents, batch_size=32, show_progress=True)

print(f"\n✅ Embeddings générés")
print(f"📐 Shape : {embeddings.shape}")
print(f"💾 Taille : {embeddings.nbytes / 1024 / 1024:.2f} MB")

🔄 Initialisation du provider : mistral
✅ Mistral client initialisé (dimension: 1024)
🧠 Génération de 497 embeddings...
⚠️ Cela peut prendre plusieurs minutes



🧠 Mistral Embeddings:   0%|          | 0/16 [00:00<?, ?it/s]


✅ Embeddings générés
📐 Shape : (497, 1024)
💾 Taille : 1.94 MB


## 5️⃣ Validation


In [5]:
# Vérifier la qualité des embeddings
print("✅ Validation des embeddings :")
print(f"  Nombre : {len(embeddings)}")
print(f"  Dimension : {embeddings.shape[1]}")
print(f"  Type : {embeddings.dtype}")
print(f"  Valeurs nulles : {np.isnan(embeddings).sum()}")
print(f"  Min : {embeddings.min():.4f}")
print(f"  Max : {embeddings.max():.4f}")
print(f"  Moyenne : {embeddings.mean():.4f}")

✅ Validation des embeddings :
  Nombre : 497
  Dimension : 1024
  Type : float32
  Valeurs nulles : 0
  Min : -0.1210
  Max : 0.1220
  Moyenne : -0.0002


## 6️⃣ Test de Similarité


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

# Tester sur les 5 premiers documents
sample_embeddings = embeddings[:5]
similarity_matrix = cosine_similarity(sample_embeddings)

print("🔍 Matrice de similarité (5 premiers documents) :")
print("\nIndices des documents :")
for i in range(5):
    print(f"{i}: {documents[i]['title'][:50]}")

print("\n📊 Similarités :")
print(similarity_matrix.round(3))

🔍 Matrice de similarité (5 premiers documents) :

Indices des documents :
0: Formation Civique et Citoyenne "Cadre de vie et ar
1: "CONSTRUCTEURS DE DEMAIN, Découverte ludique des m
2: "Viv(r)e l'architecture, un architecte dans la cla
3: Exposition "Hors Site (mais pas hors sol)"
4: Devenez conducteur de bus - POEI RTM

📊 Similarités :
[[1.    0.872 0.902 0.85  0.84 ]
 [0.872 1.    0.924 0.873 0.838]
 [0.902 0.924 1.    0.876 0.836]
 [0.85  0.873 0.876 1.    0.814]
 [0.84  0.838 0.836 0.814 1.   ]]


## 7️⃣ Export


In [7]:
# Créer le dossier embeddings
embeddings_dir = PROCESSED_DATA_DIR / "embeddings"
embeddings_dir.mkdir(exist_ok=True)

# Export embeddings (numpy)
embeddings_path = embeddings_dir / "embeddings.npy"
np.save(embeddings_path, embeddings)

# Export métadonnées
metadata_path = embeddings_dir / "metadata.json"
metadata = {
    "provider": settings.embedding_provider,
    "model_name": settings.embedding_model,
    "embedding_dim": embeddings.shape[1],
    "num_documents": len(embeddings),
    "document_ids": [doc["id"] for doc in documents],
}

with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"💾 Embeddings : {embeddings_path}")
print(f"💾 Métadonnées : {metadata_path}")
print(f"\n✅ Export terminé !")
print(f"\n➡️ Prochaine étape : Notebook 04 - Construction de l'index FAISS")

💾 Embeddings : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/embeddings/embeddings.npy
💾 Métadonnées : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/embeddings/metadata.json

✅ Export terminé !

➡️ Prochaine étape : Notebook 04 - Construction de l'index FAISS
